In [1]:
import tensorflow as tf
import numpy as np

tf.set_random_seed(777)

In [2]:

sample = " if you want you"
idx2char = list(set(sample))
char2idx = {c: i for i, c in enumerate(idx2char)}

# hyper parameters (sample만 바꾸게 되면 나머지 parameter를 다 바꿀 수 있도록)
dic_size = len(char2idx) # RNN input size(one hot size)
hidden_size = len(char2idx) # RNN output size
rnn_hidden_size = len(char2idx) # RNN output size
num_classes = len(char2idx) # final output size (RNN or softmax, etc.)
batch_size = 1 # one sample data, one batch
sequence_length = len(sample) - 1 # number of lstm unfolding (unit #)


sample_idx = [char2idx[c] for c in sample] # char to index
x_data = [sample_idx[:-1]] # X data sample (0 ~ n-1)
y_data = [sample_idx[1:]] # Y label sample (1 ~ n)

X = tf.placeholder(tf.int32, [None, sequence_length]) # X data
Y = tf.placeholder(tf.int32, [None, sequence_length]) # Y label

"""
one_hot만들때 dimension차이가 있기에 주의할 것
"""
X_one_hot = tf.one_hot(X, num_classes) # one hot: 1 -> 0 1 0 0 0 0 0 0 0 0

cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_hidden_size, 
                                    state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(
    cell, X_one_hot, initial_state=initial_state, dtype=tf.float32)

# FC layer
X_for_fc = tf.reshape(outputs, [-1, hidden_size])
outputs = tf.contrib.layers.fully_connected(outputs, num_classes, activation_fn=None)

# reshape out for sequence_loss
outputs = tf.reshape(outputs, [batch_size, sequence_length, num_classes])

weights = tf.ones([batch_size, sequence_length])
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)
loss = tf.reduce_mean(sequence_loss)
train = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

prediction = tf.argmax(outputs, axis=2)

In [3]:
# traning and results

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(1000):
        l, _ = sess.run([loss, train], feed_dict={X: x_data, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_data})
        # print char using dic
        result_str = [idx2char[c] for c in np.squeeze(result)]
        print(i, 'loss: ', l, 'Prediction:', ''.join(result_str))

0 loss:  2.30427 Prediction: yyyyoo o o   o 
1 loss:  2.17917 Prediction: yyyyou       ou
2 loss:  2.03012 Prediction: yyyyou      uou
3 loss:  1.82425 Prediction: yyyyo  oan  you
4 loss:  1.55246 Prediction: yyyyou aant you
5 loss:  1.25225 Prediction: yyyyou want you
6 loss:  0.966964 Prediction: yy you want you
7 loss:  0.724996 Prediction: yy you want you
8 loss:  0.518058 Prediction: yf you want you
9 loss:  0.373755 Prediction: yf you want you
10 loss:  0.265109 Prediction: if you want you
11 loss:  0.187001 Prediction: if you want you
12 loss:  0.129728 Prediction: if you want you
13 loss:  0.0888709 Prediction: if you want you
14 loss:  0.0605884 Prediction: if you want you
15 loss:  0.041454 Prediction: if you want you
16 loss:  0.0286302 Prediction: if you want you
17 loss:  0.0199648 Prediction: if you want you
18 loss:  0.0140683 Prediction: if you want you
19 loss:  0.0100843 Prediction: if you want you
20 loss:  0.00742156 Prediction: if you want you
21 loss:  0.00564277 

169 loss:  0.000172368 Prediction: if you want you
170 loss:  0.000171605 Prediction: if you want you
171 loss:  0.00017081 Prediction: if you want you
172 loss:  0.000170016 Prediction: if you want you
173 loss:  0.000169309 Prediction: if you want you
174 loss:  0.000168546 Prediction: if you want you
175 loss:  0.000167775 Prediction: if you want you
176 loss:  0.000167036 Prediction: if you want you
177 loss:  0.000166266 Prediction: if you want you
178 loss:  0.000165542 Prediction: if you want you
179 loss:  0.000164827 Prediction: if you want you
180 loss:  0.000164104 Prediction: if you want you
181 loss:  0.000163405 Prediction: if you want you
182 loss:  0.00016269 Prediction: if you want you
183 loss:  0.000161967 Prediction: if you want you
184 loss:  0.000161307 Prediction: if you want you
185 loss:  0.000160576 Prediction: if you want you
186 loss:  0.000159885 Prediction: if you want you
187 loss:  0.000159202 Prediction: if you want you
188 loss:  0.00015855 Prediction:

334 loss:  9.33201e-05 Prediction: if you want you
335 loss:  9.3042e-05 Prediction: if you want you
336 loss:  9.2756e-05 Prediction: if you want you
337 loss:  9.2446e-05 Prediction: if you want you
338 loss:  9.21917e-05 Prediction: if you want you
339 loss:  9.18898e-05 Prediction: if you want you
340 loss:  9.16117e-05 Prediction: if you want you
341 loss:  9.13335e-05 Prediction: if you want you
342 loss:  9.10554e-05 Prediction: if you want you
343 loss:  9.07852e-05 Prediction: if you want you
344 loss:  9.04832e-05 Prediction: if you want you
345 loss:  9.0229e-05 Prediction: if you want you
346 loss:  8.99747e-05 Prediction: if you want you
347 loss:  8.96489e-05 Prediction: if you want you
348 loss:  8.93866e-05 Prediction: if you want you
349 loss:  8.91324e-05 Prediction: if you want you
350 loss:  8.88781e-05 Prediction: if you want you
351 loss:  8.86079e-05 Prediction: if you want you
352 loss:  8.83456e-05 Prediction: if you want you
353 loss:  8.80755e-05 Prediction: 

503 loss:  5.90941e-05 Prediction: if you want you
504 loss:  5.8951e-05 Prediction: if you want you
505 loss:  5.88239e-05 Prediction: if you want you
506 loss:  5.86809e-05 Prediction: if you want you
507 loss:  5.85378e-05 Prediction: if you want you
508 loss:  5.84027e-05 Prediction: if you want you
509 loss:  5.82915e-05 Prediction: if you want you
510 loss:  5.81405e-05 Prediction: if you want you
511 loss:  5.79895e-05 Prediction: if you want you
512 loss:  5.78862e-05 Prediction: if you want you
513 loss:  5.77034e-05 Prediction: if you want you
514 loss:  5.75763e-05 Prediction: if you want you
515 loss:  5.74571e-05 Prediction: if you want you
516 loss:  5.73299e-05 Prediction: if you want you
517 loss:  5.71869e-05 Prediction: if you want you
518 loss:  5.70756e-05 Prediction: if you want you
519 loss:  5.69167e-05 Prediction: if you want you
520 loss:  5.67816e-05 Prediction: if you want you
521 loss:  5.66703e-05 Prediction: if you want you
522 loss:  5.65193e-05 Predictio

669 loss:  4.14362e-05 Prediction: if you want you
670 loss:  4.13647e-05 Prediction: if you want you
671 loss:  4.12932e-05 Prediction: if you want you
672 loss:  4.11899e-05 Prediction: if you want you
673 loss:  4.11104e-05 Prediction: if you want you
674 loss:  4.10468e-05 Prediction: if you want you
675 loss:  4.09673e-05 Prediction: if you want you
676 loss:  4.08879e-05 Prediction: if you want you
677 loss:  4.07925e-05 Prediction: if you want you
678 loss:  4.0713e-05 Prediction: if you want you
679 loss:  4.06495e-05 Prediction: if you want you
680 loss:  4.05859e-05 Prediction: if you want you
681 loss:  4.05144e-05 Prediction: if you want you
682 loss:  4.0419e-05 Prediction: if you want you
683 loss:  4.03395e-05 Prediction: if you want you
684 loss:  4.0268e-05 Prediction: if you want you
685 loss:  4.01726e-05 Prediction: if you want you
686 loss:  4.01091e-05 Prediction: if you want you
687 loss:  4.00217e-05 Prediction: if you want you
688 loss:  3.9966e-05 Prediction: 

833 loss:  3.09224e-05 Prediction: if you want you
834 loss:  3.08906e-05 Prediction: if you want you
835 loss:  3.07952e-05 Prediction: if you want you
836 loss:  3.07396e-05 Prediction: if you want you
837 loss:  3.06919e-05 Prediction: if you want you
838 loss:  3.06681e-05 Prediction: if you want you
839 loss:  3.05965e-05 Prediction: if you want you
840 loss:  3.05409e-05 Prediction: if you want you
841 loss:  3.05091e-05 Prediction: if you want you
842 loss:  3.04455e-05 Prediction: if you want you
843 loss:  3.0382e-05 Prediction: if you want you
844 loss:  3.03661e-05 Prediction: if you want you
845 loss:  3.03184e-05 Prediction: if you want you
846 loss:  3.02866e-05 Prediction: if you want you
847 loss:  3.01992e-05 Prediction: if you want you
848 loss:  3.01595e-05 Prediction: if you want you
849 loss:  3.01118e-05 Prediction: if you want you
850 loss:  3.00482e-05 Prediction: if you want you
851 loss:  3.00085e-05 Prediction: if you want you
852 loss:  2.99449e-05 Predictio

999 loss:  2.39449e-05 Prediction: if you want you


In [6]:
print(char2idx)

{'y': 6, 'w': 1, ' ': 0, 'i': 2, 'u': 7, 'n': 9, 'o': 8, 't': 3, 'f': 4, 'a': 5}
